In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import levy


In [ ]:
import random

In [ ]:
import numpy as np
import random
from scipy.stats import levy

class FPA:
    """
    Implementação do Algoritmo de Polinização por Flores (FPA).
    
    Atributos:
        n (int): Número de nós.
        m (int): Número de flores.
        D_matrix (np.ndarray): Matriz de distâncias.
        s_prob (float): Probabilidade de troca.
    """

    def __init__(self, dist_matrix, switch_prob, number_nodes, number_flowers):
        """
        Inicializa o FPA com os parâmetros fornecidos.

        Args:
            dist_matrix (list de listas de float): Matriz de distâncias.
            switch_prob (float): Probabilidade de troca.
            number_nodes (int): Número de nós.
            number_flowers (int): Número de flores.
        """
        self.n = number_nodes
        self.m = number_flowers
        self.D_matrix = np.array(dist_matrix, dtype=np.float64)
        self.s_prob = switch_prob

    def biotic_pollination(self, costs):
        """
        Realiza a polinização biótica usando uma distribuição de voo de Levy.

        Args:
            costs (list de tuplas): Lista de tuplas onde cada tupla contém o custo e o nó correspondente.

        Returns:
            int: O nó selecionado com base na polinização biótica.
        """
        costs_array = np.array(costs)
        probs = []
        r = random.random()
        for i in range(len(costs)):
            lev = levy.cdf(np.sum(costs_array[:i+1, 0], axis=0)) - levy.cdf(np.sum(costs_array[:i, 0], axis=0))
            probs.append((lev, costs_array[i][1]))
        # cumulative_probs =[]
        # for prob_index in range(len(probs)):
        #     if prob_index ==0:
        #         cumulative_probs.append(probs[prob_index])
        #     else:
        #         cumulative = sum([i[0] for i in probs[:prob_index+1]])
        #         cumulative_probs.append((cumulative,probs[prob_index][1]))

        min_val = float('inf')
        min_node = None
        for i in probs:
            if abs(i[0] - r) < min_val:
                min_node = i
                min_val = abs(i[0] - r)
        return min_node

    def abiotic_pollination(self, costs, radius, previous_node):
        """
        Realiza a polinização abiótica com base em restrições de distância.

        Args:
            costs (list de tuplas): Lista de tuplas onde cada tupla contém o custo e o nó correspondente.
            radius (float): Raio dentro do qual a polinização abiótica pode ocorrer.
            previous_node (int): O nó anterior.

        Returns:
            list: Uma lista contendo um booleano indicando sucesso e o nó selecionado.
        """
        new_costs = []
        for i in costs:
            if self.D_matrix[int(previous_node)][int(i[1])] <= radius:
                new_costs.append(i)
        if len(new_costs) == 0:
            return [False, None]
        probs = []
        costs_array = np.array(new_costs)
        r = random.random()
        for i in range(len(new_costs)):
            categorical = np.sum(costs_array[:i+1, 0])
            probs.append((categorical, costs_array[i][1]))
        min_val = float('inf')
        min_node = None
        for i in probs:
            if abs(i[0] - r) <= min_val:
                min_node = i
                min_val = abs(i[0] - r)
        return [True, min_node]

    def initialize_solutions(self):
        # """
        # Inicializa as soluções aleatoriamente.

        # Returns:
        #     list: Uma lista de soluções inicializadas.
        # """
        # sol = []
        # for i in range(self.m):
        #     sol.append(random.sample(range(self.n), self.n))
        # return sol
        """
        Inicializa as soluções aleatoriamente.

        Returns:
            list: Uma lista de soluções inicializadas.
        """
        sol = []
        for i in range(self.m):
            sol.append([])
            for j in range(self.n):
                if j == 0:
                    random_init = random.randint(0, self.n - 1)
                    sol[i].append(random_init)
                else:
                    sol[i].append(0)
        return sol

    def calculate_distance(self, solution):
        """
        Calcula a distância total de uma solução dada.

        Args:
            solution (list): Uma lista representando o caminho da solução.

        Returns:
            float: A distância total da solução.
        """
        distance = 0
        for i in range(1, self.n):
            distance += self.D_matrix[int(solution[i-1]), int(solution[i])]
        return distance

    def calculate_cost(self, vertices, node, cost_map):
        """
        Calcula os custos para cada vértice a partir de um nó dado.

        Args:
            vertices (list): Lista de vértices.
            node (int): O nó atual.
            cost_map (np.ndarray): Mapa de custos baseado na matriz de distâncias.

        Returns:
            list: Lista de tuplas onde cada tupla contém o custo normalizado e o vértice correspondente.
        """
        current_costs = [(cost_map[int(node)][int(vertice)], vertice) for vertice in vertices]
        A = sum(cost[0] for cost in current_costs)
        for i in range(len(current_costs)):
            if A == 0:
                current_costs[i] = (0, current_costs[i][1])
            else:
                current_costs[i] = (current_costs[i][0] / A, current_costs[i][1])
        current_costs.sort(key=lambda x: x[1], reverse=True)
        return current_costs

    def main_loop(self,qtd_matrix,radius):
        """
        Executa o loop principal do FPA.

        Returns:
            dict: Um dicionário contendo a melhor sequência, a melhor distância e o tempo.
        """
        cost_map = 1 / self.D_matrix
        cost_map[np.isinf(cost_map)] = 0
        solution = self.initialize_solutions()
        for i in range(self.m):
            vertices = [k for k in range(0, self.n) if k != solution[i][0]]
            for j in range(1, self.n):
                previous_node = solution[i][j-1]
                costs = self.calculate_cost(vertices, previous_node, cost_map)
                r = random.random()
                if r >= self.s_prob:
                    current_node = self.biotic_pollination(costs)
                else:
                    success, current_node = self.abiotic_pollination(costs, radius, previous_node)
                    if not success:
                        current_node = self.biotic_pollination(costs)
                solution[i][j] = current_node[1]
                vertices.remove(int(current_node[1]))
        distances = [(self.calculate_distance(s), s) for s in solution]
        distances.sort(key=lambda x: x[0])
        best_distance, best_solution = distances[0]
        best_solution = [int(i) for i in best_solution]
        time=0
        for i in qtd_matrix:
            time += i*(0.5/60)
        time+= best_distance/5
        return {'sequence': best_solution, 'distance': best_distance, 'time': time}




In [ ]:
def two_opt(path, distances, qtd_matrix, max_iterations=100):
    
    # Flag para controlar se houve melhoria no caminho
    improved = True
    # Contador para controlar o número de iterações
    iteration_count = 0

    # Loop continua enquanto houver melhoria e o número de iterações for menor que o máximo
    while improved and iteration_count < max_iterations:
        improved = False
        # Testa todas as possíveis trocas de segmentos não consecutivos
        for i in range(1, len(path) - 2):
            for j in range(i + 1, len(path) - 1):
                if j - i == 1:  # Ignora se i e j são consecutivos
                    continue
                # Verifica se a troca resulta em uma distância menor
                if distances[path[i - 1]][path[i]] + distances[path[j]][path[j + 1]] > distances[path[i - 1]][path[j]] + distances[path[i]][path[j + 1]]:
                    # Realiza a troca invertendo o segmento
                    path[i:j + 1] = path[i:j + 1][::-1]
                    improved = True

        # Incrementa o contador de iterações
        iteration_count += 1

    # Recalcula distância e tempo após a otimização 2-opt
    dist = 0
    time = 0
    for k in range(1, len(path)):
        dist += distances[path[k - 1]][path[k]]
        time += distances[path[k - 1]][path[k]] / 5 + (0.5 / 60) * qtd_matrix[path[k - 1]]

    # Retorna o caminho otimizado, a distância total e o tempo estimado
    return {'sequence': path, 'distance': dist, 'time': time}


In [ ]:
%pip install haversine

In [ ]:
from haversine import haversine, Unit
data = pd.read_csv('dados_tratados_clusterizados_10_10.csv')
grouped=data.groupby('cluster_leiturista')
solutions =[]
for _, group in grouped:
   
    clusters = group.groupby('cluster_dia')
    solution_leiturista = []
    solution_distance = 0

    for label, cluster in clusters:
        coords = cluster[['LATITUDE', 'LONGITUDE']].to_numpy()   


        n = len(coords)
        distance_matrix = np.zeros((n, n))
        qtd_matrix = cluster['QUANTIDADE_HIDROMETROS'].values
        qtd_matrix=np.insert(qtd_matrix,[0],[0],axis=0)
        for i in range(n):
            for j in range(i + 1, n):
                distance_matrix[i, j] = distance_matrix[j, i] = haversine(coords[i], coords[j], unit=Unit.KILOMETERS)
        fpa = FPA(distance_matrix, 0.05, len(distance_matrix), 30)
        solution = fpa.main_loop(qtd_matrix,0.07)

        print(f"Melhor distância: {solution['distance']}")
        print(f"Melhor solução: {solution['sequence']}")
        print(f"Melhor tempo: {solution['time']}")
        # solution= two_opt(solution['sequence'],distance_matrix,qtd_matrix,200)
        # print(f"Melhor distância c/ 2-opt: {solution['distance']}")
        # print(f"Melhor solução c/ 2-opt: {solution['sequence']}")
        # print(f"Melhor tempo c/ 2-opt: {solution['time']}")
        solution_leiturista.append({f"sequence_dia_{cluster['cluster_dia'].values[0]}":solution['sequence'],f"distance_dia_{cluster['cluster_dia'].values[0]}":solution['distance'],f"tempo_dia_{cluster['cluster_dia'].values[0]}":solution['time']})
    solutions.append({f"leiturista{group['cluster_leiturista'].values[0]}":solution_leiturista})
    

pd.DataFrame(solutions).to_json('./tsp_FPA_teste_AMOSTRA_TOTAL_dia10_l_10_novo_s_retorno_init_for_node__cost_reverseT-cost[1]0506.json',indent=2)

In [ ]:
sq=pd.read_json("./tsp_FPA_teste_AMOSTRA_TOTAL_dia10_l_10_novo_s_retorno_init_for_node__cost_reverseT-cost[1]0506.json").values[0][0][0]['sequence_dia_10']

In [ ]:
data2 = []

In [ ]:
pd.Series(sq).duplicated().value_counts()

In [ ]:
data_c = pd.read_csv('dados_tratados_clusterizados_10_10.csv')


In [ ]:
for i in sq:
    data2.append(data_c.iloc[int(i),2:4].values)

In [ ]:
d =0
for i in range(1,a2.shape[0]):
     d +=haversine(a2[i-1], a2[i], unit=Unit.KILOMETERS)
print(d)

In [ ]:
df = pd.DataFrame(a2,columns=["lat","lon"])

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lon = df["lon"].values,
    lat = df["lat"].values,
    marker = {'size': 10}))


fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': -43.3, 'lat': -23},
        'style': "open-street-map",
        'center': {'lon': -43.3, 'lat': -23},
        'zoom': 1})
fig.show()


In [ ]:
a2= np.array(data2)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(a2[:,1],a2[:,0])
plt.plot(a2[:,1],a2[:,0])
